# Niezbalansowana klasyfikacja

## Wstęp

Klasyfikacja obrazów to proces przypisywania etykiety do obrazu na podstawie jego zawartości. Przykładowo, chcielibyśmy, aby nasz program komputerowy mógł rozpoznawać, czy na obrazie jest kot, pies, samochód, samolot czy może coś zupełnie innego. W dzisiejszych czasach popularnym narzędziem do rozpoznawania obrazów są tzw. sieci konwolucyjne (CNN).

Sieci konwolucyjne są rodzajem sieci neuronowych, które potrafią analizować i rozpoznawać wzorce w danych wizualnych. 

W przypadku klasyfikacji obrazów, sieć konwolucyjna składa się z kilku warstw, w tym konwolucyjnych i poolingowych. Warstwy konwolucyjne służą do ekstrakcji cech z obrazu, następnie za pomocą warstw poolingowych zmniejszamy wymiary danych, a na końcu wykorzystujemy warstwy w pełni połączone do klasyfikacji obrazu.

Progresywne zmniejszanie warstw pozwala sieciom rozpoznawać coraz to bardziej abstrakcyjne cechy jako złożenie wielu pomniejszych cech np. ptak to coś co ma dziób i jest opierzone. Dziób z kolei to np. ostry kształt o żółtawym kolorze a opierzenie oznacza pokrycie dużą ilością małych kresek.

### Zadanie

Zaimplementuj klasyfikator `YourCnnClassifier`, rozpoznający i klasyfikujący obrazki na dwie klasy. Powinna być to konwolucyjna sieć neuronowa napisana z użyciem pakietu `pytorch`.

Twoimi danymi w tym zadaniu są obrazki w formacie \*.jpg o wymiarze 224 x 224. Obrazki te dzielą się na dwie kategorie: *normal* oraz *onion*, którym przypisano odpowiednio etykiety 0 i 1.

Obrazki z klasy *normal* przedstawiają jasnoszare figury na czarnym tle. Natomiast obrazki z klasy *onion* różnią się tym, że mają dodane ciemnoszare pasma tworzące warstwy w środku jasnoszarych figur, co upodabnia je do cebuli. Wszystkie obrazki są dodatkowo zaszumione.

Publicznym interfejsem klasy `YourCnnClassifier` muszą być dwie metody ([class methods](https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner) dokładnie rzecz biorąc):
- `load` - ma wczytać parametry modelu z pliku `cnn-classifier.pth`. Tego będziemy używać podczas testowania twojego rozwiązania
- `create_with_training` - ma wytrenować model i zapisać jego parametry do pliku `cnn-classifier.pth`.

### Kryterium oceny

Twoje rozwiązanie oceniane będzie na podstawie skuteczności klasyfikacji

$$
\mathrm{score}(accuracy) = \begin{cases}
    0 & \text{jeżeli } accuracy < 0.5 \\
    (accuracy - 0.5) * 2 & \text{w.p.p.}
\end{cases}
$$

Powyższe kryterium, klasa abstrakcyjna opisująca interfejs modelu oraz ładowanie danych, są zaimplementowane poniżej przez nas. Jednocześnie podany jest przykład trywialnego klasyfikatora, który zawsze twierdzi, że próbka jest normalna. Tym samym podczas testowania na zbalansowanym zbiorze testowym otrzymuje on 0 pkt.

### Pliki zgłoszeniowe

1. Ten notebook
2. Plik zawierający wagi modelu o nazwie `cnn-classifier.pth`

**Uwaga:** Zbiór danych treningowych, który dostarczamy, jest niezbalansowany, natomiast Twoje rozwiązanie testowane będzie na zbalansowanym zbiorze, aby metryka `accuracy` była miarodajna. Weź to pod uwagę podczas tworzenia swojego modelu.

### Ograniczenia

- Ewaluacja twojego rozwiązania (bez treningu, flaga `FINAL_EVALUATION_MODE` ustawiona na `True`) na 50 przykładach testowych powinna trwać nie dłużej niż 2 minuty na Google Colab **bez** GPU.
- Wykonanie skryptu na Google Colab **bez** GPU z flagą `FINAL_EVALUATION_MODE` ustawioną na `False` powinno wytrenować model i wygenerować plik z wagami w nie więcej niż 15 minut.
- Rozmiar pliku `cnn-classifier.pth` nie powienien przekroczyć 35MB.

## Ewaluacja

Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` możesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających. 

Za to zadanie możesz zdobyć pomiędzy 0 i 1 punktów. Liczba punktów, które zdobędziesz będzie równa wartości `score`, wyliczonej na zbiorze testowym.

# Kod startowy

In [13]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [14]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

import abc
import os

import glob
import gdown
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import zipfile

## Ładowanie danych

In [15]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

GDRIVE_DATA = [
    ("1bR87z7ZI3gLK0vAGkyr_cnVGZ9P9bO7A", "train_data.zip"),
    ("1TA0lWnjJCv3lyRMML4JNHsJz3RJ-TUwZ", "valid_data.zip"),
]

def download_data():
    for file_id, zip_name in GDRIVE_DATA:
        folder_name = zip_name.split(".")[0]
        if not os.path.exists(folder_name):
            url = f'https://drive.google.com/uc?id={file_id}'
            gdown.download(url, output=zip_name, quiet=True)
            with zipfile.ZipFile(zip_name, 'r') as zip_ref:
                    zip_ref.extractall(folder_name)
            os.remove(zip_name)

download_data()

class ImageDataset(torch.utils.data.Dataset):
    """Implementacja abstrakcji zbioru danych z torch'a."""
    def __init__(self, dataset_type: str):
        self.filelist = glob.glob(f"{dataset_type}_data/*")
        self.labels   = [0 if "normal" in path else 1 for path in self.filelist]

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx) -> tuple[torch.Tensor, int]:
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = torchvision.transforms.functional.to_tensor(plt.imread(self.filelist[idx])[:,:,0])
        label = self.labels[idx]
        return image, label
    
    def loader(self, **kwargs) -> torch.utils.data.DataLoader:
        """
        Stwórz, `DataLoader`'a dla aktualnego zbioru danych.

        Wszystkie `**kwargs` zostaną przekazane do konstruktora `torch.utils.data.DataLoader`.
        `DataLoader`'y w skrócie to abstrakcja ładowania danych usdostępniająca wygodny interfejs.
        Możesz dowiedzieć się o nich więcej tutaj: https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
        """
        return torch.utils.data.DataLoader(self, **kwargs)
    
train_dataset: ImageDataset = ImageDataset("train")
valid_dataset: ImageDataset = ImageDataset("valid")

## Kod z kryterium oceniającym

In [16]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

def accuracy_to_points(accuracy: float) -> float:
    """Oblicz wynik na podstawie celności predykcji."""
    return (round(accuracy, 2) - 0.5) * 2 if accuracy > 0.5 else 0.0

def grade(model):
    """Oceń ile punktów otrzyma aktualne zadanie."""
    model.eval()
    test_loader = valid_dataset.loader()
    correct = 0
    total = 0
    with torch.no_grad():
        for [images, labels] in test_loader:
            outputs = model(images).squeeze()
            incorrect_indices = torch.where((outputs > 0.5).int() != labels)[0]
            correct += len(labels) - len(incorrect_indices)
            total += len(labels)
        accuracy = correct / total if total != 0 else 0
        if not FINAL_EVALUATION_MODE:
            print(f"Accuracy: {int(round(accuracy, 2) * 100)}%")
        return accuracy_to_points(accuracy)

## Publiczny interfejs rozwiązania

Tylko tego wymagamy od Twojej klasy, w Twoim rozwiązaniu możesz modyfikować swoją klasę do woli dodając nowe metody oraz atrybuty klasy - cokolwiek co będzie Ci potrzebne do rozwiązania zadania.

In [17]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

class CnnClassifier(torch.nn.Module, abc.ABC):
    MODEL_PATH: str = "cnn-classifier.pth"
    
    @classmethod
    def load(cls):
        """Załaduj model z pliku."""
        model = cls()
        model.load_state_dict(torch.load(cls.MODEL_PATH))
        return model


    @classmethod
    @abc.abstractmethod
    def create_with_training(cls):
        """Zapisz model do pliku."""
        pass

## Przykładowe rozwiązanie
Poniżej prezentujemy proste rozwiązanie, które w oczywisty sposób nie jest optymalne. Służy temu, aby było wiadomo w jaki sposób ma działać cały notatnik.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

if not FINAL_EVALUATION_MODE:
    class DummyCnnClassifier(CnnClassifier):
        def forward(self, x):
            batch_size, *_ = x.shape
            return torch.zeros(batch_size)
        
        @classmethod
        def create_with_training(cls):
            return cls()
    
    dummy_model = DummyCnnClassifier.create_with_training()
    print(f"DummyCnnClassifier -- Ocena: {grade(dummy_model)} pkt")

    del dummy_model
    del DummyCnnClassifier

# Twoje Rozwiązanie

In [ ]:
# Hubert Jastrzębski - V LO Kraków

from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.nn.functional as F

np.random.seed(42)
torch.manual_seed(42)

In [20]:
class YourCnnClassifier(CnnClassifier):
    def __init__(self):
        super(YourCnnClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=3, padding="same")
        self.conv2 = nn.Conv2d(4, 8, kernel_size=3, padding="same")
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8 * 56 * 56, 1)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 8 * 56 * 56) 
        x = torch.sigmoid(self.fc1(x)) 
        return x.squeeze(1)

    @classmethod
    def create_with_training(cls):
        # hyperparameters
        learning_rate = 0.001
        epochs_num = 10

        model = YourCnnClassifier()
        loss_fn = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        # creating data loader
        class_counts = [train_dataset.labels.count(0), train_dataset.labels.count(1)]
        num_samples = len(train_dataset.labels)
        class_weights = [num_samples / class_counts[i] for i in range(len(class_counts))]
        weights = [class_weights[train_dataset.labels[i]] for i in range(num_samples)]
        sampler = WeightedRandomSampler(torch.DoubleTensor(weights), num_samples)
        trainloader = DataLoader(train_dataset, batch_size=32, sampler=sampler)

        # training the model
        for epoch in range(epochs_num): 

            """running_loss = 0.0"""
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = loss_fn(outputs, labels.float())
                loss.backward()
                optimizer.step()

        # saving the model
        torch.save(model.state_dict(), cls.MODEL_PATH)

        return model

In [21]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

your_model = YourCnnClassifier.load() if FINAL_EVALUATION_MODE else YourCnnClassifier.create_with_training()

# Ewaluacja

In [22]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

def evaluate_model(model):
    """Oceń ile punktów otrzyma aktualne zadanie."""
    return grade(model)

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

if not FINAL_EVALUATION_MODE:
    print(f"YourCnnClassifier -- Ocena: {evaluate_model(your_model):.2f} pkt")